In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
#from scipy import interpolate
import networkx as nx

import pandas as pd
from sklearn.cluster import KMeans

from sknetwork.hierarchy import cut_straight, cut_balanced, Paris
from sknetwork.visualization import svg_dendrogram, svg_graph

import landscape as ld
from networkx.algorithms import approximation as apx

from IPython.display import SVG

from typing import List, Tuple

In [3]:
%matplotlib inline

# Init

In [4]:
import os
from pathlib import Path
ENV_PATH = Path(os.environ.get("CONDA_PREFIX"))
SOLVER_DIR = ENV_PATH / "envs/landscape_steiner/bin"

# Optimization Model

In [5]:
from pyomo.environ import (
    ConcreteModel,
    Var,
    Set,
    Binary,
    Objective,
    SolverFactory,
    minimize,
)

In [25]:
def pyomo_demo(verbose=False):
    m = ConcreteModel()

    # sets
    m.NODES = Set(initialize=[0,1,2])

    # variables
    m.x = Var(m.NODES, domain=Binary)
    
    # obj
    m.obj = Objective(expr=sum(m.x[node] for node in m.NODES), sense=minimize)
    
    # results
    results = SolverFactory("cbc", executable=SOLVER_DIR / "cbc").solve(m, tee=verbose)
    return (
        m.obj.expr(),
        m,
    )

In [28]:
value, m = pyomo_demo()